In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '' 
import jax

jax.config.update('jax_enable_x64', True)
jax.config.update("jax_default_matmul_precision", "highest")

from mace.calculators import mace_mp,mace_off
from ase import build
from fennol import FENNIX
import yaml
import jax.numpy as jnp
import numpy as np  
from fennol.ase import FENNIXCalculator
from fennol.utils import AtomicUnits as au
from copy import deepcopy
from mace import modules as torch_modules
import e3nn as e3nn_torch
import torch
from fennol.utils.activations import normalize_activation

from ase.units import Hartree
print(au.EV)
print(Hartree)

e3nn_torch.__version__

27.211386024367243
27.211386024367243


'0.4.4'

In [2]:
atoms = build.molecule('H2O')
model_name = "mace_off_large"
atoms.set_positions([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0]])
calc = mace_off(model="large", default_dtype="float64", device='cpu')
atoms.calc = calc
print(atoms.get_potential_energy())

torch_model_mp = calc.models[0].cpu()
species_order = torch_model_mp.atomic_numbers.tolist()
print("species_order:",species_order)
print(max(species_order))
num_species = len(species_order)
print(num_species)

scale = torch_model_mp.scale_shift.scale.item()
shift = torch_model_mp.scale_shift.shift.item()
print("scale",scale/au.EV)
print("shift",shift/au.EV)

avg_num_neigh = torch_model_mp.interactions[0].avg_num_neighbors
print("avg_num_neighbors",avg_num_neigh)

print(torch_model_mp)




Using MACE-OFF23 MODEL for MACECalculator with /home/pthomas/.cache/mace/MACE-OFF23_large.model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
-2080.921473483096
species_order: [1, 6, 7, 8, 9, 15, 16, 17, 35, 53]
53
10
scale 0.040001721591467386
shift 0.0
avg_num_neighbors 18.41771125793457
ScaleShiftMACE(
  (node_embedding): LinearNodeEmbeddingBlock(
    (linear): Linear(10x0e -> 224x0e | 2240 weights)
  )
  (radial_embedding): RadialEmbeddingBlock(
    (bessel_fn): BesselBasis(r_max=5.0, num_basis=8, trainable=False)
    (cutoff_fn): PolynomialCutoff(p=5.0, r_max=5.0)
  )
  (spherical_harmonics): SphericalHarmonics()
  (atomic_energies_fn): AtomicEnergiesBlock(energies=[-13.5720, -1030.5672, -1486.3750, -2043.9337, -2715.3185, -9287.4071, -10834.4845, -12522.6493, -70045.2839, -8102.5246])
  (interactions): ModuleList(
    (0): RealAgnosticInteractionBlock(
      (linear_up): Linear(224x0e -> 224x0e | 50176 weights)
      (

In [3]:
r_max = torch_model_mp.radial_embedding.bessel_fn.r_max.item()
num_basis = torch_model_mp.radial_embedding.bessel_fn.bessel_weights.numel()
p = torch_model_mp.radial_embedding.cutoff_fn.p.item()
max_ell = torch_model_mp.spherical_harmonics._lmax
num_inter = len(torch_model_mp.products)
corr = len(torch_model_mp.products[0].symmetric_contractions.contractions[0].weights)+1
print(torch_model_mp.readouts[1].non_linearity.acts[0].cst)
print(f"{r_max=},{num_basis=},{p=},{max_ell=},{num_inter=},{num_species=},{avg_num_neigh=},{corr=}")
torch_model = torch_modules.ScaleShiftMACE(
    atomic_inter_scale=scale,
    atomic_inter_shift=shift,
    r_max=r_max,
    num_bessel=num_basis,
    num_polynomial_cutoff=p,
    max_ell=max_ell,
    interaction_cls_first=torch_modules.RealAgnosticInteractionBlock,
    interaction_cls=torch_modules.RealAgnosticResidualInteractionBlock,
    num_interactions=num_inter,
    num_elements=num_species,
    hidden_irreps=e3nn_torch.o3.Irreps("224x0e+224x1o+224x2e"),
    MLP_irreps="16x0e",
    avg_num_neighbors=avg_num_neigh,
    correlation=corr,
    atomic_energies=torch_model_mp.atomic_energies_fn.atomic_energies.clone().detach(),
    atomic_numbers=torch_model_mp.atomic_numbers.clone().detach(),
    gate=torch.nn.SiLU(),
).double()
print(torch_model)

1.6791767923989418
r_max=5.0,num_basis=8,p=5.0,max_ell=3,num_inter=2,num_species=10,avg_num_neigh=18.41771125793457,corr=3


/home/pthomas/miniconda3/envs/mace/lib/python3.10/site-packages/mace/modules/models.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "atomic_numbers", torch.tensor(atomic_numbers, dtype=torch.int64)
/home/pthomas/miniconda3/envs/mace/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/pthomas/miniconda3/envs/mace/lib/python3.10/site-packages/mace/modules/blocks.py:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(atomic_

ScaleShiftMACE(
  (node_embedding): LinearNodeEmbeddingBlock(
    (linear): Linear(10x0e -> 224x0e | 2240 weights)
  )
  (radial_embedding): RadialEmbeddingBlock(
    (bessel_fn): BesselBasis(r_max=5.0, num_basis=8, trainable=False)
    (cutoff_fn): PolynomialCutoff(p=5.0, r_max=5.0)
  )
  (spherical_harmonics): SphericalHarmonics()
  (atomic_energies_fn): AtomicEnergiesBlock(energies=[-13.5720, -1030.5672, -1486.3750, -2043.9337, -2715.3185, -9287.4071, -10834.4845, -12522.6493, -70045.2839, -8102.5246])
  (interactions): ModuleList(
    (0): RealAgnosticInteractionBlock(
      (linear_up): Linear(224x0e -> 224x0e | 50176 weights)
      (conv_tp): TensorProduct(224x0e x 1x0e+1x1o+1x2e+1x3o -> 224x0e+224x1o+224x2e+224x3o | 896 paths | 896 weights)
      (conv_tp_weights): FullyConnectedNet[8, 64, 64, 64, 896]
      (linear): Linear(224x0e+224x1o+224x2e+224x3o -> 224x0e+224x1o+224x2e+224x3o | 200704 weights)
      (skip_tp): FullyConnectedTensorProduct(224x0e+224x1o+224x2e+224x3o x 10x0

/home/pthomas/miniconda3/envs/mace/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/pthomas/miniconda3/envs/mace/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/pthomas/miniconda3/envs/mace/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.w

In [4]:
torch_model.load_state_dict(torch_model_mp.state_dict())
calc = mace_off(model="large", default_dtype="float64", device='cpu')
calc.models[0] = torch_model
atoms.calc = calc
print(calc.get_potential_energy(atoms,force_consistent=True))

Using MACE-OFF23 MODEL for MACECalculator with /home/pthomas/.cache/mace/MACE-OFF23_large.model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
-2080.921473483096


In [5]:
model_params = yaml.load(open(model_name+".yaml"), Loader=yaml.FullLoader)
model_fennol = FENNIX(**model_params,rng_key=jax.random.PRNGKey(1)) 

In [6]:
variables = deepcopy(model_fennol.variables)
nn_energy_params = variables["params"]["nn_energy"]

for k in nn_energy_params.keys():
    print(k)
    if isinstance(nn_energy_params[k],dict):
        for kk in nn_energy_params[k].keys():
            print("   ",kk)


SymmetricContraction_0
    w1_0e
    w1_1o
    w1_2e
    w2_0e
    w2_1o
    w2_2e
    w3_0e
    w3_1o
    w3_2e
SymmetricContraction_1
    w1_0e
    w2_0e
    w3_0e
hidden_linear_readout_last
    w[0,0] 224x0e,16x0e
linear_contraction_0
    w[0,0] 224x0e,224x0e
    w[1,1] 224x1o,224x1o
    w[2,2] 224x2e,224x2e
linear_contraction_1
    w[0,0] 224x0e,224x0e
linear_dn_0
    w[0,0] 224x0e,224x0e
    w[1,1] 224x1o,224x1o
    w[2,2] 224x2e,224x2e
    w[3,3] 224x3o,224x3o
linear_dn_1
    w[0,0] 672x0e,224x0e
    w[1,1] 1120x1o,224x1o
    w[2,2] 1120x2e,224x2e
    w[3,3] 896x3o,224x3o
linear_readout_0
    w[0,0] 224x0e,1x0e
linear_readout_last
    w[0,0] 16x0e,1x0e
linear_up_0
    w[0,0] 224x0e,224x0e
linear_up_1
    w[0,0] 224x0e,224x0e
    w[1,1] 224x1o,224x1o
    w[2,2] 224x2e,224x2e
radial_network_0
    Dense_0
    Dense_1
    Dense_2
    Dense_3
radial_network_1
    Dense_0
    Dense_1
    Dense_2
    Dense_3
skip_tp_0
    w[0,0] 224x0e,224x0e
    w[1,1] 224x1o,224x1o
    w[2,2] 224x2e,2

In [7]:
def linear_torch_to_jax(linear):
    return {
        f"w[{ins.i_in},{ins.i_out}] {linear.irreps_in[ins.i_in]},{linear.irreps_out[ins.i_out]}": jnp.asarray(
            w.data
        )
        for i, ins, w in linear.weight_views(yield_instruction=True)
    }

def skip_tp_torch_to_jax(tp):
    return {
        f"w[{ins.i_in1},{ins.i_out}] {tp.irreps_in1[ins.i_in1]},{tp.irreps_out[ins.i_out]}": jnp.moveaxis(
            jnp.asarray(w.data), 1, 0
        )/num_species**0.5
        for i, ins, w in tp.weight_views(yield_instruction=True)
    }

In [8]:
## get initial species encodings
species_encodings = np.zeros(nn_energy_params["species_encoding"].shape,dtype=np.float64)
species_encodings_torch = torch_model.node_embedding.linear.weight.detach().numpy().reshape(num_species,-1)
print(species_encodings.shape,species_encodings_torch.shape)
assert species_encodings.shape[1] == species_encodings_torch.shape[1]
factor = num_species**-0.5
for i,z in enumerate(species_order):
    species_encodings[z,:] = species_encodings_torch[i,:]*factor
    # print(f"{z}: {species_encodings[z]}")
            
nn_energy_params["species_encoding"] = jnp.array(species_encodings,dtype=jnp.float64)
print(species_encodings[1,:10])
print(species_encodings_torch[0,:10])
print(torch_model.node_embedding.linear.instructions)

(120, 224) (10, 224)
[ 0.00068487 -0.00089906 -0.00012659 -0.00030575  0.00172688 -0.00110494
  0.00195346 -0.00072814 -0.00045683  0.01436365]
[ 0.00216575 -0.00284308 -0.00040032 -0.00096685  0.00546088 -0.00349414
  0.00617739 -0.00230257 -0.00144461  0.04542185]
[Instruction(i_in=0, i_out=0, path_shape=(10, 224), path_weight=0.31622776601683794)]


In [9]:
# species energy shift

print(len(model_fennol.variables["params"]["energy_shift"]["constant"]))
print(len(torch_model.atomic_energies_fn.atomic_energies))
energy_shift = np.zeros(len(model_fennol.variables["params"]["energy_shift"]["constant"]),dtype=np.float64)
for i,z in enumerate(species_order):
    # print(f"{z}: {model_fennol.variables['params']['energy_shift']['constant'][z]} {torch_model.atomic_energies_fn.atomic_energies[i].data/au.EV}")
    energy_shift[z] = torch_model.atomic_energies_fn.atomic_energies[i].data/au.EV
variables["params"]["energy_shift"]["constant"] = jnp.array(energy_shift,dtype=jnp.float64)
print(variables["params"]["energy_shift"]["constant"])
    

119
10
[ 0.00000000e+00 -4.98760510e-01  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -3.78726451e+01 -5.46232751e+01
 -7.51131784e+01 -9.97861162e+01  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -3.41305920e+02
 -3.98159964e+02 -4.60198876e+02  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.57411672e+03
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -2.97762289e+02  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  

In [10]:
# skip tp 0
for layer in range(2):
    print(f'skip_tp_{layer}')
    skip_params_torch = skip_tp_torch_to_jax(
        torch_model.interactions[layer].skip_tp
    )
    print(skip_params_torch.keys())
    print(nn_energy_params[f"skip_tp_{layer}"].keys())
    for k in nn_energy_params[f'skip_tp_{layer}'].keys():
        print(skip_params_torch[k].shape)
        print(nn_energy_params[f'skip_tp_{layer}'][k].shape)
        skip_params = np.zeros_like(nn_energy_params[f'skip_tp_{layer}'][k])
        assert skip_params.shape[1:] == nn_energy_params[f'skip_tp_{layer}'][k].shape[1:]
        for i,z in enumerate(species_order):
            skip_params[z,:,:] = np.array(skip_params_torch[k][i,:,:])

        nn_energy_params[f'skip_tp_{layer}'][k] = jnp.array(skip_params,dtype=jnp.float64)

# # skip tp 1
# print()
# print("skip tp 1")
# skip_params_torch = skip_tp_torch_to_jax(
#         torch_model.interactions[1].skip_tp
#     )
# print(skip_params_torch.keys())
# print(nn_energy_params["skip_tp_1"].keys())

# print(skip_params_torch['w[0,0] 128x0e,128x0e'].shape)
# print(nn_energy_params["skip_tp_1"]['w[0,0] 128x0e,128x0e'].shape)
# skip_params = np.zeros_like(nn_energy_params["skip_tp_1"]['w[0,0] 128x0e,128x0e'])
# assert skip_params.shape[1:] == nn_energy_params["skip_tp_1"]['w[0,0] 128x0e,128x0e'].shape[1:]
# for i,z in enumerate(species_order):
#     skip_params[z,:,:] = np.array(skip_params_torch['w[0,0] 128x0e,128x0e'][i,:,:])

# nn_energy_params["skip_tp_1"]['w[0,0] 128x0e,128x0e'] = jnp.array(skip_params,dtype=jnp.float64)

skip_tp_0
dict_keys(['w[0,0] 224x0e,224x0e', 'w[1,1] 224x1o,224x1o', 'w[2,2] 224x2e,224x2e', 'w[3,3] 224x3o,224x3o'])
dict_keys(['w[0,0] 224x0e,224x0e', 'w[1,1] 224x1o,224x1o', 'w[2,2] 224x2e,224x2e', 'w[3,3] 224x3o,224x3o'])
(10, 224, 224)
(120, 224, 224)
(10, 224, 224)
(120, 224, 224)
(10, 224, 224)
(120, 224, 224)
(10, 224, 224)
(120, 224, 224)
skip_tp_1
dict_keys(['w[0,0] 224x0e,224x0e'])
dict_keys(['w[0,0] 224x0e,224x0e'])
(10, 224, 224)
(120, 224, 224)


In [11]:
# linear up
for i in range(2):
    print()
    print(f"linear_up_{i}")
    up_params_torch = linear_torch_to_jax(torch_model.interactions[i].linear_up)
    print(up_params_torch.keys())
    print(nn_energy_params[f"linear_up_{i}"].keys())
    for k in up_params_torch.keys():
        print(k)
        print(up_params_torch[k].shape)
        print(nn_energy_params[f"linear_up_{i}"][k].shape)
        assert up_params_torch[k].shape == nn_energy_params[f"linear_up_{i}"][k].shape
        nn_energy_params[f"linear_up_{i}"][k] = jnp.array(up_params_torch[k],dtype=jnp.float64)

# linear down
for i in range(2):
    print()
    print(f"linear_dn_{i}")
    up_params_torch = linear_torch_to_jax(torch_model.interactions[i].linear)
    print(up_params_torch.keys())
    print(nn_energy_params[f"linear_dn_{i}"].keys())
    for k in up_params_torch.keys():
        print(k)
        print(up_params_torch[k].shape)
        print(nn_energy_params[f"linear_dn_{i}"][k].shape)
        assert up_params_torch[k].shape == nn_energy_params[f"linear_dn_{i}"][k].shape
        nn_energy_params[f"linear_dn_{i}"][k] = jnp.array(up_params_torch[k],dtype=jnp.float64)

# contraction linear
for i in range(2):
    print()
    print(f"linear_contraction_{i}")
    up_params_torch = linear_torch_to_jax(torch_model.products[i].linear)
    print(up_params_torch.keys())
    print(nn_energy_params[f"linear_contraction_{i}"].keys())
    for k in up_params_torch.keys():
        print(k)
        print(up_params_torch[k].shape)
        try:
            print(nn_energy_params[f"linear_contraction_{i}"][k].shape)
            assert up_params_torch[k].shape == nn_energy_params[f"linear_contraction_{i}"][k].shape
            nn_energy_params[f"linear_contraction_{i}"][k] = jnp.array(up_params_torch[k],dtype=jnp.float64)
        except KeyError:
            print(f"key {k} not found in nn_energy_params[f'linear_contraction_{i}']")



linear_up_0
dict_keys(['w[0,0] 224x0e,224x0e'])
dict_keys(['w[0,0] 224x0e,224x0e'])
w[0,0] 224x0e,224x0e
(224, 224)
(224, 224)

linear_up_1
dict_keys(['w[0,0] 224x0e,224x0e', 'w[1,1] 224x1o,224x1o', 'w[2,2] 224x2e,224x2e'])
dict_keys(['w[0,0] 224x0e,224x0e', 'w[1,1] 224x1o,224x1o', 'w[2,2] 224x2e,224x2e'])
w[0,0] 224x0e,224x0e
(224, 224)
(224, 224)
w[1,1] 224x1o,224x1o
(224, 224)
(224, 224)
w[2,2] 224x2e,224x2e
(224, 224)
(224, 224)

linear_dn_0
dict_keys(['w[0,0] 224x0e,224x0e', 'w[1,1] 224x1o,224x1o', 'w[2,2] 224x2e,224x2e', 'w[3,3] 224x3o,224x3o'])
dict_keys(['w[0,0] 224x0e,224x0e', 'w[1,1] 224x1o,224x1o', 'w[2,2] 224x2e,224x2e', 'w[3,3] 224x3o,224x3o'])
w[0,0] 224x0e,224x0e
(224, 224)
(224, 224)
w[1,1] 224x1o,224x1o
(224, 224)
(224, 224)
w[2,2] 224x2e,224x2e
(224, 224)
(224, 224)
w[3,3] 224x3o,224x3o
(224, 224)
(224, 224)

linear_dn_1
dict_keys(['w[0,0] 672x0e,224x0e', 'w[1,1] 1120x1o,224x1o', 'w[2,2] 1120x2e,224x2e', 'w[3,3] 896x3o,224x3o'])
dict_keys(['w[0,0] 672x0e,224x0e', 'w[

In [12]:
# readout block
print(f"linear_readout_0")
up_params_torch = linear_torch_to_jax(torch_model.readouts[0].linear)
print(up_params_torch.keys())
print(nn_energy_params[f"linear_readout_0"].keys())
for k in up_params_torch.keys():
    print(k)
    print(up_params_torch[k].shape)
    print(nn_energy_params[f"linear_readout_0"][k].shape)
    assert up_params_torch[k].shape == nn_energy_params[f"linear_readout_0"][k].shape
    nn_energy_params[f"linear_readout_0"][k] = jnp.array(up_params_torch[k],dtype=jnp.float64)

print()
print(f"hidden_linear_readout_last")
up_params_torch = linear_torch_to_jax(torch_model.readouts[1].linear_1)
print(up_params_torch.keys())
print(nn_energy_params[f"hidden_linear_readout_last"].keys())
for k in up_params_torch.keys():
    print(k)
    print(up_params_torch[k].shape)
    print(nn_energy_params[f"hidden_linear_readout_last"][k].shape)
    assert up_params_torch[k].shape == nn_energy_params[f"hidden_linear_readout_last"][k].shape
    nn_energy_params[f"hidden_linear_readout_last"][k] = jnp.array(up_params_torch[k],dtype=jnp.float64)


f,silu_norm = normalize_activation(jax.nn.silu,return_scale=True)
print("silu_norm",silu_norm,torch_model.readouts[1].non_linearity.acts[0].cst)
cst = torch_model.readouts[1].non_linearity.acts[0].cst / silu_norm

print()
print(f"linear_readout_last")
up_params_torch = linear_torch_to_jax(torch_model.readouts[1].linear_2)
print(up_params_torch.keys())
print(nn_energy_params[f"linear_readout_last"].keys())
for k in up_params_torch.keys():
    print(k)
    print(up_params_torch[k].shape)
    print(nn_energy_params[f"linear_readout_last"][k].shape)
    assert up_params_torch[k].shape == nn_energy_params[f"linear_readout_last"][k].shape
    nn_energy_params[f"linear_readout_last"][k] = cst*jnp.array(up_params_torch[k],dtype=jnp.float64)


linear_readout_0
dict_keys(['w[0,0] 224x0e,1x0e'])
dict_keys(['w[0,0] 224x0e,1x0e'])
w[0,0] 224x0e,1x0e
(224, 1)
(224, 1)

hidden_linear_readout_last
dict_keys(['w[0,0] 224x0e,16x0e'])
dict_keys(['w[0,0] 224x0e,16x0e'])
w[0,0] 224x0e,16x0e
(224, 16)
(224, 16)
silu_norm 1.6765620560798837 1.6791767923989418

linear_readout_last
dict_keys(['w[0,0] 16x0e,1x0e'])
dict_keys(['w[0,0] 16x0e,1x0e'])
w[0,0] 16x0e,1x0e
(16, 1)
(16, 1)


In [13]:
# radial net

for i in range(2):
    print("radial net",i)
    print(nn_energy_params[f"radial_network_{i}"].keys())
    w  = torch_model.interactions[i].conv_tp_weights.layer0.weight.detach().numpy()
    w = w/w.shape[0]**0.5
    print(w.shape,nn_energy_params[f"radial_network_{i}"]["Dense_0"]["kernel"].shape)
    assert w.shape == nn_energy_params[f"radial_network_{i}"]["Dense_0"]["kernel"].shape
    nn_energy_params[f"radial_network_{i}"]["Dense_0"]["kernel"] = jnp.array(w,dtype=jnp.float64)

    cst = torch_model.interactions[i].conv_tp_weights.layer0.act.cst/ silu_norm
    print("cst",cst)

    w  = cst*torch_model.interactions[i].conv_tp_weights.layer1.weight.detach().numpy()
    w = w/w.shape[0]**0.5
    print(w.shape,nn_energy_params[f"radial_network_{i}"]["Dense_1"]["kernel"].shape)
    assert w.shape == nn_energy_params[f"radial_network_{i}"]["Dense_1"]["kernel"].shape
    nn_energy_params[f"radial_network_{i}"]["Dense_1"]["kernel"] =  jnp.array(w,dtype=jnp.float64)

    cst = torch_model.interactions[i].conv_tp_weights.layer0.act.cst/ silu_norm
    print("cst",cst)

    w  = cst*torch_model.interactions[i].conv_tp_weights.layer2.weight.detach().numpy()
    print(w.shape,nn_energy_params[f"radial_network_{i}"]["Dense_2"]["kernel"].shape)
    w = w/w.shape[0]**0.5
    assert w.shape == nn_energy_params[f"radial_network_{i}"]["Dense_2"]["kernel"].shape
    nn_energy_params[f"radial_network_{i}"]["Dense_2"]["kernel"] =  jnp.array(w,dtype=jnp.float64)

    cst = torch_model.interactions[i].conv_tp_weights.layer0.act.cst/ silu_norm
    print("cst",cst)

    w  = cst*torch_model.interactions[i].conv_tp_weights.layer3.weight.detach().numpy()
    w = w/w.shape[0]**0.5
    print(w.shape,nn_energy_params[f"radial_network_{i}"]["Dense_3"]["kernel"].shape)
    assert w.shape == nn_energy_params[f"radial_network_{i}"]["Dense_3"]["kernel"].shape
    nn_energy_params[f"radial_network_{i}"]["Dense_3"]["kernel"] =  jnp.array(w,dtype=jnp.float64)
    print()


radial net 0
dict_keys(['Dense_0', 'Dense_1', 'Dense_2', 'Dense_3'])
(8, 64) (8, 64)
cst 1.001559582187594
(64, 64) (64, 64)
cst 1.001559582187594
(64, 64) (64, 64)
cst 1.001559582187594
(64, 896) (64, 896)

radial net 1
dict_keys(['Dense_0', 'Dense_1', 'Dense_2', 'Dense_3'])
(8, 64) (8, 64)
cst 1.001559582187594
(64, 64) (64, 64)
cst 1.001559582187594
(64, 64) (64, 64)
cst 1.001559582187594
(64, 3808) (64, 3808)



In [14]:
from mace.tools.cg import U_matrix_real
from e3nn_jax import reduced_tensor_product_basis
from e3nn import o3
import e3nn_jax

def get_reordering(order,ir_out):
    # torch 
    irreps_in = o3.Irreps("1x0e + 1x1o + 1x2e + 1x3o")
    irrep_out = o3.Irreps("1x"+ir_out)
    U_matrix = U_matrix_real(irreps_in=irreps_in, irreps_out=irrep_out, correlation=order)[-1].numpy()
    U_matrix_abs = np.abs(U_matrix)

    # jax
    irreps_in = e3nn_jax.Irreps("1x0e + 1x1o + 1x2e + 1x3o")
    irrep_out = e3nn_jax.Irrep(ir_out)
    U = reduced_tensor_product_basis([irreps_in] * order, keep_ir=[irrep_out]).list[-1]
    U = np.moveaxis(U,-1,0)
    U_abs = np.abs(U)

    index = []
    signs = []
    for i in range(U.shape[-1]):
        diffs = []
        for k in range(U_matrix.shape[-1]):
            diff = np.max(np.abs(U_abs[...,i]-U_matrix_abs[...,k]))
            diffs.append(diff)
        k = np.argmin(diffs)
        diff = diffs[k]
        if diff>1.e-6:
            print(i,k,diff)
            raise ValueError("diff too large")
        
        index.append(k)
        diff = np.max(np.abs(U[...,i]-U_matrix[...,k]))
        if diff < 1.e-6:
            signs.append(1.)
        else:
            diff = np.max(np.abs(U[...,i]+U_matrix[...,k]))
            if diff < 1.e-6:
                print(i,k,"global sign differ")
                signs.append(-1.)
            else:
                raise ValueError("local sign differ")
    print(order,ir_out,len(index),U.shape[-1],len(signs))
    return np.array(index),np.array(signs)
    return np.arange(U.shape[-1])

In [15]:
ir_layers = [["0e","1o","2e"],["0e"]]
for layer in [0,1]:
    print()
    print("layer",layer,"->",list(nn_energy_params[f"SymmetricContraction_{layer}"].keys()))

    for ir in ir_layers[layer]:
        l=int(ir[0])
        corr = len(torch_model.products[layer].symmetric_contractions.contractions[l].weights)+1
        print("   ir",ir,"l",l,"corr",corr)

        w=torch_model.products[layer].symmetric_contractions.contractions[l].weights_max.detach().cpu().numpy()

        print(f"     w{corr}_{ir}",w.shape,nn_energy_params[f"SymmetricContraction_{layer}"][f"w{corr}_{ir}"].shape) 
        assert w.shape[1:] == nn_energy_params[f"SymmetricContraction_{layer}"][f"w{corr}_{ir}"].shape[1:]

        # reorder weights
        index,signs = get_reordering(corr,ir)
        w = w[:,index,:]*signs[None,:,None]


        # last order
        skip_params = np.zeros_like(nn_energy_params[f"SymmetricContraction_{layer}"][f"w{corr}_{ir}"])
        for i,z in enumerate(species_order):
            skip_params[z,:,:] = w[i,:,:]
        nn_energy_params[f"SymmetricContraction_{layer}"][f"w{corr}_{ir}"] = jnp.asarray(skip_params,dtype=jnp.float64)

        print(np.abs(w).mean())

        for c in range(corr-1):
            order = corr-c-1
            w=torch_model.products[layer].symmetric_contractions.contractions[l].weights[c].detach().cpu().numpy()
            print(f"     w{order}_{ir}",w.shape,nn_energy_params[f"SymmetricContraction_{layer}"][f"w{order}_{ir}"].shape)
            assert w.shape[1:] == nn_energy_params[f"SymmetricContraction_{layer}"][f"w{order}_{ir}"].shape[1:]

            index,signs = get_reordering(order,ir)
            w = w[:,index,:]*signs[None,:,None]

            skip_params = np.zeros_like(nn_energy_params[f"SymmetricContraction_{layer}"][f"w{order}_{ir}"])
            for i,z in enumerate(species_order):
                skip_params[z,:,:] = w[i,:,:]
            nn_energy_params[f"SymmetricContraction_{layer}"][f"w{order}_{ir}"] = jnp.asarray(skip_params,dtype=jnp.float64)
            print(np.abs(w).mean())



layer 0 -> ['w1_0e', 'w1_1o', 'w1_2e', 'w2_0e', 'w2_1o', 'w2_2e', 'w3_0e', 'w3_1o', 'w3_2e']
   ir 0e l 0 corr 3
     w3_0e (10, 23, 224) (120, 23, 224)
3 0e 23 23 23
1.455638554091279
     w2_0e (10, 4, 224) (120, 4, 224)
2 0e 4 4 4
1.3833557012728157
     w1_0e (10, 1, 224) (120, 1, 224)
1 0e 1 1 1
0.5838673742783771
   ir 1o l 1 corr 3
     w3_1o (10, 51, 224) (120, 51, 224)
17 17 global sign differ
18 18 global sign differ
35 21 global sign differ
36 22 global sign differ
44 44 global sign differ
46 46 global sign differ
3 1o 51 51 51
1.3892637638000955
     w2_1o (10, 6, 224) (120, 6, 224)
2 1o 6 6 6
1.6495189089561173
     w1_1o (10, 1, 224) (120, 1, 224)
1 1o 1 1 1
0.614487849317727
   ir 2e l 2 corr 3
     w3_2e (10, 65, 224) (120, 65, 224)
16 16 global sign differ
35 23 global sign differ
36 25 global sign differ
37 20 global sign differ
38 22 global sign differ
53 53 global sign differ
55 55 global sign differ
3 2e 65 65 65
1.3573949041483202
     w2_2e (10, 7, 224) (120, 7,

In [16]:
atoms.calc = calc  
print("MACE torch",atoms.get_potential_energy())

variables["params"]["nn_energy"] = nn_energy_params  
model_fennol.variables = variables 

model_fennol.save(model_name+".fnx")




MACE torch -2080.921473483096


In [17]:
model_fennol_load = FENNIX.load(model_name+".fnx")
# print("##############################")
# print("Restored model")
# for k in model_fennol_load.variables["params"]["nn_energy"].keys():
#     print(k)
#     if isinstance(nn_energy_params[k],dict):
#         for kk in nn_energy_params[k].keys():
#             print("   ",kk)
calc_fennol = FENNIXCalculator(model=model_fennol_load,atoms=atoms,use_float64=True)
atoms.calc = calc_fennol
print("MACE fennol",atoms.get_potential_energy())

MACE fennol -2080.921473483096


In [18]:
species = atoms.get_atomic_numbers()
coordinates = atoms.get_positions()
output = model_fennol_load(coordinates=coordinates,species=species)
print(output.keys())
print(output["nn_energy"]*au.EV)
# print(output["nn_energy_node_feats"][0])

dict_keys(['batch_index', 'coordinates', 'energy_shift', 'graph', 'natoms', 'nn_energy', 'nn_energy_node_feats', 'species'])
[-4.11289955 -2.86547566 -2.86547566]


In [19]:
node_feats_torch = calc.get_descriptors(atoms)
# print(node_feats_torch[0])
print(calc.get_property("node_energy",atoms=atoms))

[-4.11289955 -2.86547566 -2.86547566]
